In [105]:
import pandas as pd
from tqdm import tqdm
import json
import requests
from pathlib import Path
import re
import random
import numpy as np
import time

In [106]:
def get_publisher_download_url(crossref_resp):
    # search for 'text-mining' in the intended-application field
    text_mining = []
    for link in crossref_response.json()['link']:
        if re.search(r'text-mining',link['intended-application']):
            text_mining.append(re.search('text-mining', link['intended-application']))
        else:
            text_mining.append(None)
            
    # find the index of match obects in crossref link information
    publisher_api_idxs = np.where(np.array(text_mining) != None)[0].tolist()
    # wiley_api_idxs
    links = [crossref_response.json()['link'][i] for i in publisher_api_idxs]
    # for now use the first link if there are more than one link
    full_text_link = links[0]['URL']
    return full_text_link

In [108]:
def download_pdf(doi, dest_folder, content):
    doi_suffix = str(doi.split('/', 1)[1:]).strip("[']")
    doi_suffix = re.sub(r"""([()/\\*,"': ?;<>])""", '_._', doi_suffix)
    filename = dest_folder + '/' + str(doi_suffix)+'.pdf'
    with open(filename, mode = 'wb') as f:
        f.write(content)

In [109]:
with open('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/code/cpet_articles/gathering/full-text_download_code/wiley/wiley_config.json') as config_file:
    wiley_token = json.load(config_file)['api_key']

In [110]:
pdf_file_paths = list(Path('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/pdfs').rglob('*/*.pdf'))
pdfs = [path.stem for path in pdf_file_paths]
len(pdfs)

13091

In [111]:
articles = pd.read_csv('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/unpaywall/unpaywall_info.csv')
wiley_oa_articles = articles[(articles['publisher'] == 'Wiley') & (articles['is_oa'] == True)].drop_duplicates().reset_index(drop=True)

In [112]:
re_doi_suffix = re.compile(r'(?<=\d/).*')
wiley_oa_articles['doi_suffix'] = wiley_oa_articles['doi'].apply(lambda x: re_doi_suffix.search(x).group())

In [113]:
full_texts_to_download = [x for x in wiley_oa_articles['doi_suffix'].tolist() if x not in pdfs]
merge = pd.merge(pd.DataFrame({'doi_suffix': full_texts_to_download}), wiley_oa_articles, how='inner', on='doi_suffix')
merge.shape

(297, 50)

In [114]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:101.0) Gecko/20100101 Firefox/101.0'
accept = 'application/vnd.citationstyles.csl+json, application/vnd.crossref.unixref+xml'
dest_folder = '/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/pdfs/wiley_oa'

In [124]:
# n = random.randint(0, merge.shape[0])
# row = merge.loc[n,:]
# doi = row['doi']
# doi

'10.1002/clc.4960101111'

In [125]:
# doi = row['doi']
# out = {'doi': doi}
# doi_url = 'https://doi.org/' + str(doi)
# crossref_headers = {'Accept': accept, 'User-Agent': user_agent}
# try:
#     crossref_response = requests.get(
#                     url = doi_url,
#                     headers=crossref_headers,
#                     allow_redirects=True,
#                     verify=True)
#     out.update({'CR_status_code': crossref_response.status_code})

#     if crossref_response.status_code == 200:
#         full_text_url = get_publisher_download_url(crossref_response)
#         publisher_headers = {
#             'User-Agent': user_agent,
#             'Wiley-TDM-Client-Token': wiley_token,
#             # 'CR-TDM-Rate-Limit': '1000',
#             # 'CR-TDM-Rate-Limit-Remaining': '76',
#             # 'CR-TDM-Rate-Limit-Reset': '1378072800'
#         }
#         publisher_response = requests.get(full_text_url, headers = publisher_headers, allow_redirects=True)
#         out.update({'publisher_status_code': publisher_response.status_code})
#         print(publisher_response.status_code)
# except Exception as e:
#     print(e)
#     out.update({'error': e})


200


In [126]:
log = []

In [127]:
for idx, row in tqdm(merge.iterrows(), total=merge.shape[0]):
    doi = row['doi']
    out = {'doi': doi}
    doi_url = 'https://doi.org/' + str(doi)
    crossref_headers = {'Accept': accept, 'User-Agent': user_agent}
    try:
        crossref_response = requests.get(
                        url = doi_url,
                        headers=crossref_headers,
                        allow_redirects=True,
                        verify=True)
        out.update({'CR_status_code': crossref_response.status_code})

        if crossref_response.status_code == 200:
            full_text_url = get_publisher_download_url(crossref_response)
            publisher_headers = {
                'User-Agent': user_agent,
                'Wiley-TDM-Client-Token': wiley_token,
                # 'CR-TDM-Rate-Limit': '1000',
                # 'CR-TDM-Rate-Limit-Remaining': '76',
                # 'CR-TDM-Rate-Limit-Reset': '1378072800'
            }
            publisher_response = requests.get(full_text_url, headers = publisher_headers, allow_redirects=True)
            out.update({'publisher_status_code': publisher_response.status_code})
            if publisher_response.status_code == 200:
                download_pdf(doi = doi, dest_folder=dest_folder, content=publisher_response.content)
    except Exception as e:
        print(e)
        out.update({'error': e})
    time.sleep(0.35)

    log.append(out)

 54%|█████▍    | 161/297 [05:12<07:03,  3.11s/it]

HTTPConnectionPool(host='api.wiley.com', port=80): Max retries exceeded with url: /onlinelibrary/tdm/v1/articles/10.1016%2Fj.ejheart.2005.05.002 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f8154980af0>: Failed to establish a new connection: [Errno 61] Connection refused'))


 59%|█████▉    | 175/297 [05:44<03:09,  1.55s/it]

list index out of range


 59%|█████▉    | 176/297 [05:45<02:37,  1.30s/it]

...

 78%|███████▊  | 232/297 [06:38<00:59,  1.08it/s]

HTTPConnectionPool(host='api.wiley.com', port=80): Max retries exceeded with url: /onlinelibrary/tdm/v1/articles/10.1002%2Fj.1939-4640.1987.tb00981.x (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f8154fb84c0>: Failed to establish a new connection: [Errno 61] Connection refused'))


 84%|████████▎ | 248/297 [06:56<00:49,  1.01s/it]

list index out of range


100%|██████████| 297/297 [07:42<00:00,  1.56s/it]


In [128]:
log_df = pd.DataFrame(log)
log_df['publisher_status_code'].value_counts()

503.0    117
500.0    105
200.0     58
400.0      1
Name: publisher_status_code, dtype: int64

In [130]:
# log_df[log_df['publisher_status_code'] != 200].reset_index(drop=True)

,doi,CR_status_code,publisher_status_code,error
0,10.1111/sms.14156,200,503.0,NaN
1,10.1111/sms.14130,200,503.0,NaN
2,10.1002/mus.27423,200,503.0,NaN
3,10.1111/sms.14113,200,503.0,NaN
4,10.1111/sms.14101,200,503.0,NaN
...,...,...,...,...
234,10.1002/jcsm.12786,200,503.0,NaN
235,10.1002/jcsm.12777,200,503.0,NaN
236,10.1111/sms.13377,200,500.0,NaN
237,10.1113/jphysiol.2010.204487,200,500.0,NaN


In [131]:
non200 = log_df[log_df['publisher_status_code'] != 200].reset_index(drop=True)
non200.shape

(239, 4)

In [133]:
wiley_errors = pd.merge(non200, merge, how='inner', on='doi')
wiley_errors.to_csv('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/unpaywall/wiley_oa_status_codes.csv',
                    index=False)

In [134]:
wiley_errors.shape

(239, 53)